#Setup

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Input
from sklearn.model_selection import train_test_split
import numpy as np
import os
from skimage.io import imread, imshow
from skimage.color import rgb2hsv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Preparing the Data




In [ ]:
data_dir = 'Dataset Filters/RGB/'

train = []
y_train = []
path_train = data_dir + 'Train'

classes = os.listdir(path_train)
for classe in classes:

  list_images = os.listdir(path_train + '/' + classe)

  for img in list_images:
    image = imread(path_train + '/' + classe + '/' + img)
    hsv = rgb2hsv(image)
    image = image/255
    image_input = np.dstack([(image[:,:,0]-image[:,:,1]), (image[:,:,0]-image[:,:,2]), hsv[:,:,0], hsv[:,:,1], hsv[:,:,2]])
    train.append(image_input)
    y_train.append(int(classe))

train = np.array(train)
y_train = np.array(y_train)

In [ ]:
valid = []
y_valid = []
path_valid = data_dir + 'Valid'

classes = os.listdir(path_valid)
for classe in classes:

  list_images = os.listdir(path_valid + '/' + classe)

  for img in list_images:
    image = imread(path_valid + '/' + classe + '/' + img)
    hsv = rgb2hsv(image)
    image = image/255
    image_input = np.dstack([(image[:,:,0]-image[:,:,1]), (image[:,:,0]-image[:,:,2]), hsv[:,:,0], hsv[:,:,1], hsv[:,:,2]])
    valid.append(image_input)
    y_valid.append(int(classe))

valid = np.array(valid)
y_valid = np.array(y_valid)

# Training

In [ ]:
model = tf.keras.models.Sequential()
model.add(Input(shape = (40,40,5)))
model.add(Conv2D(64, kernel_size=(5, 5), padding='same', activation='relu'))
model.add(Conv2D(96, kernel_size=(5, 5), activation='relu'))
model.add(Conv2D(1024, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=5e-5), loss= 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint('best_model', monitor='val_accuracy', verbose=1, save_best_only=True)]

In [ ]:
model.fit(train, y_train,
          validation_data=(valid, y_valid),
          callbacks=callbacks,
          batch_size = 32,
          epochs=100)

In [ ]:
model = tf.keras.models.load_model('best_model')
model.save('model_color_filter_2.h5')